In [ ]:
import pandas as pd
import cufflinks
cufflinks.go_offline()

In [ ]:
data_h5 = "../data/data.h5"

In [ ]:
title = "24g/PM10"
all_data = pd.read_hdf(data_h5, title)
metadata = pd.read_hdf(data_h5, "metadata")

In [ ]:
cities = pd.DataFrame(metadata.groupby('Miejscowość')['Kod stacji'].apply(list))
# select the cities with at least 3 stations
large_cities = cities[cities['Kod stacji'].map(len) >= 3]

In [ ]:
def get_means(all_data, city, codes, freq):
    if set(codes).intersection(set(all_data.columns)):
        return all_data.loc[:, codes].mean(1)
    else:
        return pd.Series(name=city)

In [ ]:
res = pd.DataFrame({city: get_means(all_data, city, codes, 'W') for city, codes in list(large_cities['Kod stacji'].items())}).dropna([0,1],'all')

In [ ]:
(res
 .groupby(pd.Grouper(freq='M'))
 .mean()
 .pipe(lambda df: df.loc[:,df.isnull().mean()<0.8])
 .pipe(lambda df: df.T.assign(mean=df.mean()).sort_values('mean').drop('mean',axis=1).T)
 .iplot(kind='heatmap', colorscale='reds', dimensions=(1000,1700))
)

In [ ]:
(res
 .apply(lambda x: x>35).astype(float)
 .groupby(pd.Grouper(freq='M'))
 .sum()
 .pipe(lambda df: df.loc[:,df.isnull().mean()<0.8])
 .pipe(lambda df: df.T.assign(mean=df.mean()).sort_values('mean').drop('mean',axis=1).T)
 .iplot(kind='heatmap', colorscale='reds', dimensions=(1000,1700))
)